In [1]:
from rdkit import Chem
from rdkit.Chem import rdmolops

# Load molecules from SDF string or file
suppl = Chem.SDMolSupplier('/home/calvin/code/chemprop_phd_customised/habnet/data/processed/sdf_data/rxn_3248.sdf', removeHs=False, sanitize=True)
mol_r1h = suppl[0]
mol_r2h = suppl[1]

In [2]:
import pandas as pd

rxn_feats = pd.read_csv('/home/calvin/code/chemprop_phd_customised/habnet/data/processed/target_data/target_data_sin_cos.csv')
rxn_ids = rxn_feats['rxn'].tolist()

In [3]:
len(rxn_ids)

1696

In [23]:
'rxn_1850' in rxn_ids

True

In [49]:
import os
import re

ts_dict = {}

def has_valid_ts_log(folder_path):
    for log_name in ['input.log', 'output.log', 'output.out']:
        log_path = os.path.join(folder_path, log_name)
        if os.path.exists(log_path):
            with open(log_path, 'r') as f:
                for line in reversed(f.readlines()):
                    if 'Normal termination' in line:
                        return True
    return False

def try_ts_guess_fallback(base_path):
    for tsg in ['tsg0', 'tsg1']:
        guess_path = os.path.join(base_path, 'calcs', 'TS_guesses','TS0', tsg)
        if not os.path.exists(guess_path):
            continue
        for file in os.listdir(guess_path):
            if file == 'AutoTST F.xyz':
                return os.path.join(guess_path, file)
        for file in os.listdir(guess_path):
            if 'Heuristic' in file and file.endswith('.xyz'):
                return os.path.join(guess_path, file)
    return None

base_dir = "/home/calvin/Dropbox/PersonalFolders/Calvin/"

for folder_1 in os.listdir(base_dir):
    if folder_1.endswith("Converged"):
        folder_1_path = os.path.join(base_dir, folder_1)
        for folder_2 in os.listdir(folder_1_path):
            if folder_2 in rxn_ids:
                ts0_path = os.path.join(folder_1_path, folder_2, "calcs", "TSs", "TS0")
                if os.path.exists(ts0_path):
                    subfolders = os.listdir(ts0_path)
                    ts_found = False
                    for folder_3 in subfolders:
                        if re.match(r"^conformer\d+$", folder_3) or re.match(r"^conf_opt_\d+$", folder_3):
                            full_path = os.path.join(ts0_path, folder_3)
                            if has_valid_ts_log(full_path):
                                ts_dict[folder_2] = full_path
                                ts_found = True
                                break
                    # Fallback if no conformer/conf_opt_ found
                    if not ts_found:
                        fallback_xyz = try_ts_guess_fallback(os.path.join(folder_1_path, folder_2))
                        if fallback_xyz:
                            ts_dict[folder_2] = fallback_xyz
            elif folder_2 == 'NonRMG':
                for folder_2a in os.listdir(f"/home/calvin/Dropbox/PersonalFolders/Calvin/{folder_1}/{folder_2}"):
                    if folder_2a in rxn_ids:

                        ts0_path = os.path.join(folder_1_path,folder_2, folder_2a, "calcs", "TSs", "TS0")
                        if os.path.exists(ts0_path):
                            subfolders = os.listdir(ts0_path)
                            ts_found = False
                            for folder_3 in subfolders:
                                if re.match(r"^conformer\d+$", folder_3) or re.match(r"^conf_opt_\d+$", folder_3):
                                    full_path = os.path.join(ts0_path, folder_3)
                                    if has_valid_ts_log(full_path):
                                        ts_dict[folder_2a] = full_path
                                        ts_found = True
                                        break
                            # Fallback if no conformer/conf_opt_ found
                            if not ts_found:
                                fallback_xyz = try_ts_guess_fallback(os.path.join(folder_1_path,folder_2, folder_2a))
                                if fallback_xyz:
                                    ts_dict[folder_2a] = fallback_xyz


In [50]:
len(ts_dict)

1696

In [52]:
for rxn in rxn_ids:
    if rxn not in ts_dict:
        print(rxn)

In [53]:
ts_dict_long = ts_dict.copy()


In [54]:
len(ts_dict_long)

1696

In [55]:
for rxn in rxn_ids:
    if rxn not in ts_dict_long:
        print(rxn)

In [58]:
import numpy as np

def find_archive_end(lines):
    for i in range(len(lines) - 1, -1, -1):
        if lines[i].strip().endswith("\\@") or lines[i].strip() == "@":
            return i
        # Handle wrapped case: line is a backslash, next is just '@'
        if lines[i].strip() == "\\" and i+1 < len(lines) and lines[i+1].strip() == "@":
            return i+1  # Point to the '@' line
    return None

def extract_final_xyz_archive_block(logfile: str) -> tuple[list[str], np.ndarray]:
    with open(logfile, "r") as f:
        lines = [line.strip() for line in f]

    # Step 1: Find the last line containing '\@'
    end_idx = find_archive_end(lines)
    if end_idx is None:
        print(f"WARNING: File {logfile} has no recognizable archive block end (\\@ or @) — skipping.")
        return None, None

    # Step 2: Find the last preceding line containing '\\0,1\\' or '\\0,2\\' or '\\0,3\\'
    start_idx = None
    for i in range(end_idx, -1, -1):
        if '\\0,1\\' in lines[i] or '\\0,2\\' in lines[i] or '\\0,3\\' in lines[i]:
            start_idx = i
            break
    if start_idx is None:
        raise ValueError("Could not find start of Gaussian archive coordinate block (\\0,1\\ or \\0,2\\)")

    # Step 3: Join all lines between start and end into a single string
    archive_block = "".join(lines[start_idx:end_idx + 1])

    # Step 4: Isolate the part after '\0,1' or '\0,2'
    if "\\0,1" in archive_block:
        archive_block = archive_block.split("\\0,1", 1)[-1]
    elif "\\0,2" in archive_block:
        archive_block = archive_block.split("\\0,2", 1)[-1]

    # Step 5: Split into coordinate entries
    entries = archive_block.strip("\\").split("\\")
    symbols = []
    coords = []

    for entry in entries:
        fields = entry.split(",")
        if len(fields) != 4:
            continue
        sym, x, y, z = fields
        try:
            symbols.append(sym)
            coords.append([float(x), float(y), float(z)])
        except Exception:
            continue

    if not symbols or not coords:
        raise ValueError("No atomic coordinates found in Gaussian archive block.")

    return symbols, np.array(coords, dtype=np.float64)


def parse_xyz_like_ts_file(filepath):
    atoms = []
    coords = []

    with open(filepath, 'r') as f:
        lines = f.readlines()

    if not lines:
        raise ValueError(f"{filepath} is empty or unreadable.")

    # Format A: Heuristic
    if lines[0].strip() == "Heuristic":
        atom_lines = lines[2:]  # Skip "Heuristic" and atom count
    # Format B: AutoTST
    elif len(lines) > 1 and lines[1].strip() == "AutoTST F":
        atom_lines = lines[2:]  # Skip count and "AutoTST F"
    # Format C: Generic XYZ (no special label)
    else:
        atom_lines = lines[2:]  # Generic XYZ fallback

    for line in atom_lines:
        parts = line.strip().split()
        if len(parts) != 4:
            continue  # Skip malformed lines
        symbol, x, y, z = parts
        atoms.append(symbol)
        coords.append([float(x), float(y), float(z)])

    return atoms, coords


In [59]:
extract_final_xyz_archive_block('/home/calvin/Dropbox/PersonalFolders/Calvin/Kfir_HAb_Converged/kfir_rxn_13108/calcs/TSs/TS0/conformer7/input.log')

(['O', 'C', 'C', 'C', 'C', 'O', 'H', 'C', 'C', 'N', 'O', 'N', 'H', 'H'],
 array([[ 1.46429571, -2.88826418,  0.27005411],
        [ 0.84678924, -1.8637572 ,  0.27700193],
        [ 0.0889378 , -1.33768504,  1.39571334],
        [-0.57347939, -0.83748909,  2.28096086],
        [-1.33844914, -0.21528658,  3.3570049 ],
        [-2.49749654, -0.44964642,  3.56217808],
        [-0.75155304,  0.50261694,  3.97493304],
        [ 0.50132604,  0.40771824, -1.69631094],
        [ 0.55866197,  0.84593791, -3.04854742],
        [ 0.18071576,  2.09266348, -3.04821366],
        [-0.09627471,  2.41501059, -1.77134905],
        [ 0.10664111,  1.3584926 , -0.9360323 ],
        [ 0.80298755, -1.17788009, -0.63658862],
        [ 0.84364709,  0.32789031, -3.96014246]]))

In [60]:
import re
def get_expected_atom_count(logfile: str) -> int:
    """
    Parses the Gaussian log file to extract the number of atoms from the Symbolic Z-matrix section.
    """
    atom_count = 0
    found = False
    with open(logfile, "r") as f:
        for line in f:
            if 'Symbolic Z-matrix:' in line:
                found = True
                continue
            if found:
                # Check for lines starting with element symbol (e.g., "C", "N", "O", "H", etc.)
                m = re.match(r'^\s*([A-Z][a-z]?)\b', line)
                if m:
                    atom_count += 1
                elif line.strip() == "" or not re.match(r'^[A-Z]', line.strip()):
                    break  # End of Z-matrix
    if not found or atom_count == 0:
        raise ValueError("Could not extract atom count from Symbolic Z-matrix")
    return atom_count


get_expected_atom_count('/home/calvin/Dropbox/PersonalFolders/Calvin/Kfir_HAb_Converged/kfir_rxn_13108/calcs/TSs/TS0/conformer7/input.log') 


from collections import Counter

def parse_xyz_like_ts_file(filepath):
    atoms = []
    coords = []

    with open(filepath, 'r') as f:
        lines = f.readlines()

    if not lines:
        raise ValueError(f"{filepath} is empty or unreadable.")

    # Determine format and get atomic lines
    if lines[0].strip() == "Heuristic":
        atom_lines = lines[2:]  # Skip label and count
    elif len(lines) > 1 and lines[1].strip() == "AutoTST F":
        atom_lines = lines[2:]  # Skip count and label
    else:
        atom_lines = lines[2:]  # Assume generic XYZ

    for line in atom_lines:
        parts = line.strip().split()
        if len(parts) != 4:
            continue
        symbol, x, y, z = parts
        atoms.append(symbol)
        coords.append([float(x), float(y), float(z)])

    atom_counter = Counter(atoms)

    return atoms, coords, atom_counter

In [62]:
results = {}


def try_extract_block(path_dir):
    for fname in ["input.log", "output.log", "output.out"]:
        fpath = os.path.join(path_dir, fname)
        if os.path.exists(fpath):
            result = extract_final_xyz_archive_block(fpath)
            if result:
                return result
    return None

def try_atom_count(path_dir):
    for fname in ["input.log", "output.log", "output.out"]:
        fpath = os.path.join(path_dir, fname)
        if os.path.exists(fpath):
            count = get_expected_atom_count(fpath)
            if count:
                return count
    return None
for key, value in ts_dict_long.items():
    try:
        if os.path.isdir(value):
            atom_count = try_atom_count(value)
            if atom_count is None:
                raise ValueError(f"No atom count found for {key}")

            symbols_coords = try_extract_block(value)
            if symbols_coords is None:
                raise ValueError(f"No geometry block found for {key}")

            symbols, coords = symbols_coords

            if len(symbols) != atom_count:
                raise ValueError(f"Mismatch in atom count for {key}: expected {atom_count}, found {len(symbols)}")

            atom_counter = Counter(symbols)

        elif os.path.isfile(value) and value.endswith(".xyz"):
            symbols, coords, atom_counter = parse_xyz_like_ts_file(value)
            atom_count = len(symbols)

        else:
            raise ValueError(f"Unrecognized TS path type: {value}")

        results[key] = {
            'path': value,
            'atom_count': atom_count,
            'symbols': symbols,
            'coords': coords,
            'atom_counter': atom_counter,
        }

    except Exception as e:
        print(f"Error for {key}: {e}")

In [63]:
ts_dict_long['rxn_1890']

'/home/calvin/Dropbox/PersonalFolders/Calvin/HAb_Converged/rxn_1890/calcs/TSs/TS0/conformer6'

In [64]:
len(list(results.keys()))

1696

In [65]:
results

{'kfir_rxn_13712': {'path': '/home/calvin/Dropbox/PersonalFolders/Calvin/Kfir_HAb_Converged/kfir_rxn_13712/calcs/TSs/TS0/conformer6',
  'atom_count': 14,
  'symbols': ['C',
   'O',
   'C',
   'C',
   'N',
   'H',
   'H',
   'N',
   'C',
   'C',
   'C',
   'N',
   'H',
   'H'],
  'coords': array([[ 1.38149964,  0.82179534,  0.01608109],
         [ 1.88450681,  0.29147471,  1.16155457],
         [ 1.21244313,  1.54834007,  1.27555664],
         [-0.08106911,  1.52572081,  1.93991914],
         [-1.12277861,  1.49532147,  2.43765801],
         [ 2.03720395,  1.08463818, -0.82493293],
         [ 1.85415711,  2.3994826 ,  1.52809082],
         [-1.60955379,  0.78316861, -2.96217397],
         [-1.31820935,  0.21463748, -1.99813966],
         [-0.89540409, -0.42962386, -0.77475416],
         [-0.56385743, -1.8328521 , -0.87651997],
         [-0.24734408, -2.94382666, -0.92892534],
         [ 0.19998919,  0.16623139, -0.43838783],
         [-1.56444965, -0.21228703,  0.0707279 ]]),
  'atom_co

In [68]:
import csv
import json

with open("rxn_geometries.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["rxn_id", "path", "atom_count", "symbols", "coords"])

    for rxn_id, info in results.items():
        coords = info["coords"]
        if hasattr(coords, "tolist"):
            coords = coords.tolist()

        writer.writerow([
            rxn_id,
            info["path"],
            info["atom_count"],
            json.dumps(info["symbols"]),
            json.dumps(coords),
        ])


In [ ]:
from rdkit import Chem
from rdkit.Chem import rdmolops

# Load molecules from SDF string or file
suppl = Chem.SDMolSupplier('yourfile.sdf', removeHs=False)
mol_r1h = suppl[0]
mol_r2h = suppl[1]


OSError: File error: Bad input file yourfile.sdf

In [ ]:
import json

# Get mol properties
props_r1h = json.loads(mol_r1h.GetProp("mol_properties"))
props_r2h = json.loads(mol_r2h.GetProp("mol_properties"))

d_h_idx = int([k for k, v in props_r1h.items() if v["label"] == "d_hydrogen"][0])
a_h_idx = int([k for k, v in props_r2h.items() if v["label"] == "a_hydrogen"][0])
acceptor_idx = int([k for k, v in props_r2h.items() if v["label"] == "acceptor"][0])


In [ ]:
combined = rdmolops.CombineMols(mol_r1h, mol_r2h_mod)
editable_combined = Chem.EditableMol(combined)

# d_h_idx is from mol_r1h, remains same
# acceptor_idx shifted by number of atoms in mol_r1h
offset = mol_r1h.GetNumAtoms()
acceptor_new_idx = acceptor_idx + offset

# Add bond between donor hydrogen and acceptor
editable_combined.AddBond(d_h_idx, acceptor_new_idx, order=Chem.rdchem.BondType.SINGLE)
mol_final = editable_combined.GetMol()
